In [ ]:
import h5py
import numpy as np
import os
from tqdm import tqdm
import illustris_python as il
import matplotlib.pyplot as plt
import multiprocessing as mp

basePath='./sims.TNG/TNG100-1/output'
hdf5_File_Path = './APMS_TNG100_hdf5_z12'
hdf5_files = [f for f in os.listdir(hdf5_File_Path) if f.endswith('.hdf5')]

In [ ]:
def Extremum_Num(Input_Array):
    maximum_index=list()
    minimum_index=list()
    for i in range(1,Input_Array.shape[0]-1):
        if Input_Array[i]>Input_Array[i-1] and Input_Array[i]>Input_Array[i+1]:
            maximum_index.append(i)
        elif Input_Array[i]<Input_Array[i-1] and Input_Array[i]<Input_Array[i+1]:
            minimum_index.append(i)
        elif np.isnan(Input_Array[i]):
            if (i>=2)&i<=Input_Array.shape[0]-3:
                left_index=i-1
                while(1):
                    if np.isnan(Input_Array[left_index]):
                        left_index-=1
                    else:
                        break
                right_index=i+1
                while(1):
                    if np.isnan(Input_Array[right_index]):
                        right_index+=1
                    else:
                        break
                if(left_index<1):left_index=1
                if(right_index>Input_Array.shape[0]-2):right_index=Input_Array.shape[0]-2

                left_difference=Input_Array[left_index]-Input_Array[left_index-1]
                right_difference=Input_Array[right_index]-Input_Array[right_index+1]
                if left_difference>0 and right_difference>0:
                    maximum_index.append(i)
                elif left_difference<0 and right_difference<0:
                    minimum_index.append(i)
                else:continue

    return np.array(maximum_index),np.array(minimum_index)


In [36]:
first_infall_indices=list()
first_approach_indices=list()
for file_name in tqdm(hdf5_files):
    with h5py.File(os.path.join(hdf5_File_Path, file_name), "r") as f:
        init_subindex=np.where(f['Snapshots'][()]==50)[0][0]
        distance  = np.sqrt(np.sum(f['Modified_Difference'][()]**2,axis=1))[init_subindex:]
        max_indices,min_indices=Extremum_Num(distance)

        global_max=np.nanargmax(distance[init_subindex:])
        global_min=np.nanargmin(distance[init_subindex:])

        change=distance[global_max]-distance[global_min]
        ratio=0.08
        change_limit=(change*ratio)

        extreme_indices=np.sort(np.concatenate((max_indices,min_indices)))
        to_del=list()
        skip=False
        for i in range(0,extreme_indices.shape[0]-1):
            if skip:
                skip=False
                continue
            if abs(distance[extreme_indices[i+1]]-distance[extreme_indices[i]])<change_limit:
                to_del.append(i)
                to_del.append(i+1)
                skip=True
        extreme_indices=np.delete(extreme_indices,to_del).astype(int)
        if (len(extreme_indices)<2)&(distance[-1]>distance[-2]):
            halo_index=file_name[5:9]

            if halo_index[-1]=='.':
                halo_index=halo_index[:-1]
            halo_index=int(halo_index)
            first_infall_indices.append(halo_index)
        if(len(extreme_indices)<1)&(distance[-1]<distance[-2]):
            halo_index=file_name[5:9]
            if halo_index[-1]=='.':
                halo_index=halo_index[:-1]
            halo_index=int(halo_index)
            first_approach_indices.append(halo_index)
    #plt.scatter(f['Snapshots'][()][global_max],np.sqrt(np.sum(f['Modified_Difference'][()][global_max]**2)),c='r')
    #plt.scatter(f['Snapshots'][()][global_min],np.sqrt(np.sum(f['Modified_Difference'][()][global_min]**2)),c='b')
print(len(first_infall_indices))
print(len(first_approach_indices))

100%|██████████| 4396/4396 [00:05<00:00, 741.45it/s]

591
240
